In [1]:
from kinetics_modelling.config import PROCESSED_DATA_DIR
import polars as pl
import altair as alt
alt.data_transformers.enable("vegafusion")

pl.Config(fmt_str_lengths=100)

2025-04-02 14:25:13.883 | INFO     | kinetics_modelling.config:<module>:12 - PROJ_ROOT path is: /faststorage/project/mutationalscanning/Workspaces/chcharlton/kinetics_modelling


In [22]:
q = (
    pl.scan_parquet(PROCESSED_DATA_DIR / 'ob006-run2_full')
)
q.collect().shape


(136415773, 27)

In [ ]:
q = (
    pl.scan_parquet(PROCESSED_DATA_DIR / 'ob006-run2_full')
     .select(
        'center_qual', 
        'center_seq', 
        'fn', 
        # 'center_sx_0',
        # 'center_sx_1',
        # 'center_sx_2',
        # 'center_sm_0',
        # 'center_sm_1',
        # 'center_sm_2',
        # 'center_ipd_fwd_0', 
        # 'center_ipd_rev_0', 
        # 'center_pw_fwd_0',
        # 'center_pw_rev_0',
        'center_ipd_fwd_1', 
        # 'center_ipd_rev_1', 
        # 'center_pw_fwd_1',
        # 'center_pw_rev_1',
        # 'center_ipd_fwd_2', 
        # 'center_ipd_rev_2', 
        # 'center_pw_fwd_2',
        # 'center_pw_rev_2',)
    )
    .filter(pl.col('center_qual') == [93,93,93])
    .group_by(pl.col('center_seq'))
    .agg(pl.len())
    # .unpivot(index='fn')
)
df=q.collect()
df.head()


center_seq,len
str,u32
"""TAG""",230459
"""GAT""",234288
"""GTG""",197666
"""GTT""",200202
"""TCC""",134721


In [8]:
df.select('len').min()

len
u32
16282


In [3]:

q = (
    pl.scan_parquet(PROCESSED_DATA_DIR / 'ob006-run1')
    .select(
        'center_qual', 
        'center_seq', 
        'fn', 
        'center_sx_0',
        'center_sx_1',
        'center_sx_2',
        'center_sm_0',
        'center_sm_1',
        'center_sm_2',
        'center_ipd_fwd_0', 
        'center_ipd_rev_0', 
        'center_pw_fwd_0',
        'center_pw_rev_0',
        'center_ipd_fwd_1', 
        'center_ipd_rev_1', 
        'center_pw_fwd_1',
        'center_pw_rev_1',
        'center_ipd_fwd_2', 
        'center_ipd_rev_2', 
        'center_pw_fwd_2',
        'center_pw_rev_2',
    )
    .with_columns(
        (pl.col('center_qual').list[0].alias('center_qual_0')),
        (pl.col('center_qual').list[1].alias('center_qual_1')),
        (pl.col('center_qual').list[2].alias('center_qual_2'))
    )
    .drop('center_qual')
    .group_by('center_qual_2')
    .agg(pl.all().head(1000))
    .explode(pl.exclude('center_qual_2'))
    )

df = q.collect()

In [6]:
df.write_parquet('../data/processed/ob006-run1-sample.parquet')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

X = df.select(
    'fn', 
    'center_sx_0',
    'center_sx_1',
    'center_sx_2',
    'center_sm_0',
    'center_sm_1',
    'center_sm_2',
    'center_ipd_fwd_0', 
    'center_ipd_rev_0', 
    'center_pw_fwd_0',
    'center_pw_rev_0',
    'center_ipd_fwd_1', 
    'center_ipd_rev_1', 
    'center_pw_fwd_1',
    'center_pw_rev_1',
    'center_ipd_fwd_2', 
    'center_ipd_rev_2', 
    'center_pw_fwd_2',
    'center_pw_rev_2')
y = df.select('center_qual_2')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
# reg = linear_model.Lasso(alpha=.5)
# reg.fit(X_train, y_train)